In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
from datetime import date
from datetime import datetime
import math

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [3]:
from sklearn.ensemble import RandomForestClassifier


In [4]:
tickets_bare = pd.read_csv("../Partial_Data/ticket_bare_partial.csv")

In [5]:
tickets_cleaned = pd.read_csv("../Partial_Data/ticket_cleaned_partial.csv")

In [6]:
tickets_bare[['isAttended']] = tickets_bare[['isAttended']].replace(1,0)
tickets_bare[['isAttended']] = tickets_bare[['isAttended']].replace(2,1)

In [7]:
tickets_cleaned[['isAttended']] = tickets_cleaned[['isAttended']].replace(1,0)
tickets_cleaned[['isAttended']] = tickets_cleaned[['isAttended']].replace(2,1)

In [8]:
tickets_cleaned.event_name.unique()

array(['CLT22TEN', 'CLT21LAR', 'CLT21NYJ', 'CLT21TEN', 'CLT21NE',
       'CLT21JAX', 'CLT21SEA', 'CLT22PR1', 'CLT22LAC', 'CLT21PRE',
       'CLT22HOU', 'CLT22PHI', 'CLT21TB', 'CLT22JAX', 'CLT22KC',
       'CLT22PIT', 'CLT22WAS', 'CLT22PR2', 'CLT21HOU', 'CLT21LV'],
      dtype=object)

In [9]:
tickets_bare.head()
# 1 means yes, 0 means no

,Unnamed: 0,event_name,Price,Term,SeatUniqueID,Tenure,UniqueID,isAttended,event_date_0,event_date_1,event_date_2,event_date_3,event_date_4,event_date_5,event_date_6,event_date_7,event_date_8,event_date_9,event_date_10,event_date_11
0,1184483,CLT22TEN,0.449643,0.0,244-4-20,4.0,CLT22TEN-244-4-20,1,0.000150,1.463375e-07,1.929034e-11,3.441402e-16,9.976816e-12,8.635293e-08,1.011517e-04,1.603542e-02,0.344032,0.998914,0.392526,0.020875
1,143077,CLT21LAR,0.810996,0.0,143-3-4,38.0,CLT21LAR-143-3-4,1,0.000010,4.063025e-09,2.272907e-13,2.286315e-14,6.205227e-10,2.279242e-06,1.133011e-03,7.622352e-02,0.693993,0.855129,0.142600,0.003218
2,381355,CLT21NYJ,-0.750978,0.0,636-4-15,1.0,CLT21NYJ-636-4-15,1,0.029273,2.511809e-04,2.916887e-07,4.584199e-11,9.750311e-16,4.086211e-12,4.216629e-08,5.888722e-05,0.011130,0.284686,0.985494,0.461694
3,597803,CLT21TEN,0.053322,0.0,206-13-18,4.0,CLT21TEN-206-13-18,1,0.017526,1.155214e-04,1.030519e-07,1.244116e-11,2.646158e-16,1.549923e-11,1.228614e-07,1.318050e-04,0.019136,0.376009,0.999879,0.359839
4,288379,CLT21NE,-0.226435,0.0,401-3-8,5.0,CLT21NE-401-3-8,1,0.832208,1.299226e-01,2.745041e-03,7.849176e-06,3.037458e-09,1.590770e-13,3.304797e-14,8.397145e-10,0.000003,0.001344,0.084637,0.721422


In [10]:
tickets_cleaned.head()

,Unnamed: 0,Club,Comp,Employee,Group,Group NEW,Hold,Hold Account,House,Huntington,...,event_date_2,event_date_3,event_date_4,event_date_5,event_date_6,event_date_7,event_date_8,event_date_9,event_date_10,event_date_11
0,1184483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.929034e-11,3.441402e-16,9.976816e-12,8.635293e-08,1.011517e-04,1.603542e-02,0.344032,0.998914,0.392526,0.020875
1,143077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.272907e-13,2.286315e-14,6.205227e-10,2.279242e-06,1.133011e-03,7.622352e-02,0.693993,0.855129,0.142600,0.003218
2,381355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.916887e-07,4.584199e-11,9.750311e-16,4.086211e-12,4.216629e-08,5.888722e-05,0.011130,0.284686,0.985494,0.461694
3,597803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.030519e-07,1.244116e-11,2.646158e-16,1.549923e-11,1.228614e-07,1.318050e-04,0.019136,0.376009,0.999879,0.359839
4,288379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.745041e-03,7.849176e-06,3.037458e-09,1.590770e-13,3.304797e-14,8.397145e-10,0.000003,0.001344,0.084637,0.721422


In [11]:
tickets_cleaned_extra_copy = tickets_cleaned.copy()

In [12]:
tickets_cleaned = tickets_cleaned.sample(frac= 0.05, replace= False, random_state= 1234)

### Splitting the data

In [13]:
test = tickets_cleaned[(tickets_cleaned['event_name'] == 'CLT21LV') | tickets_cleaned['event_name'] == 'CLT22HOU']
train = tickets_cleaned[(tickets_cleaned['event_name'] != 'CLT21LV') & tickets_cleaned['event_name'] != 'CLT22HOU']

- Must drop the uncodable data

In [14]:
train = train.drop(labels = ['event_name', 'UniqueID', 'Unnamed: 0', 'SeatUniqueID'], axis= 1)
test = test.drop(labels = ['event_name', 'UniqueID', 'Unnamed: 0', 'SeatUniqueID'], axis = 1)

In [15]:
y_train = train[['isAttended']]
y_test = test[['isAttended']]

In [16]:
X_train = train.drop(labels= ['isAttended'], axis = 1)
X_test = train.drop(labels = ['isAttended'], axis = 1)


In [17]:
X_train_columns = X_train.columns
X_test_columns = X_test.columns
y_train_columns = y_train.columns
y_test_columns = y_test.columns

In [18]:
#X_train = X_train.to_numpy(dtype= 'float32')
#X_test = X_test.to_numpy(dtype = 'float32')
#y_train = y_train.to_numpy(dtype = 'float32')
#y_test = y_test.to_numpy(dtype = 'float32')

In [19]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_train = y_train.ravel()
y_test = y_test.to_numpy()

In [20]:
rf_model = RandomForestClassifier(n_estimators=50, max_features="auto", random_state=123)

In [21]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=123)

In [22]:
y_pred = rf_model.predict(X_test)


In [23]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])